In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/red-wine-quality-cortez-et-al-2009")

print("Path to dataset files:", path)

In [ ]:
import numpy as np
import pandas as pd
## Visualization libraries

import matplotlib.pyplot as plt
import seaborn as sns

# ตั้งค่า Pandas เพื่อแสดงผลแบบเต็มไม่ตัดคำ
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# อ่านไฟล์และตรวจสอบ separator
import os
csv_file = os.path.join(path, "winequality-red.csv")

# ลองอ่านบรรทัดแรกเพื่อดู separator
with open(csv_file, 'r') as f:
    first_line = f.readline()
    print("บรรทัดแรกของไฟล์:", first_line[:100])
    
# ตรวจสอบว่าใช้ comma หรือ semicolon
if ',' in first_line and ';' not in first_line:
    df = pd.read_csv(csv_file)
elif ';' in first_line:
    df = pd.read_csv(csv_file, sep=';')
else:
    # ลอง pandas ตรวจสอบเอง
    df = pd.read_csv(csv_file)
    
print(f"\nอ่านข้อมูลได้แล้ว: {df.shape[0]} แถว, {df.shape[1]} คอลัมน์")
print("ชื่อคอลัมน์:", list(df.columns))

In [ ]:
df.head()

In [ ]:
# ดูข้อมูลเบื้องต้น
print("ขนาดของข้อมูล:", df.shape)
print("\nประเภทของข้อมูล:")
print(df.dtypes)
print("\nสถิติเบื้องต้น:")
df.describe()

In [ ]:
# Correlation Heatmap - ดูความสัมพันธ์ระหว่างตัวแปรทั้งหมด
plt.figure(figsize=(12, 10))
correlation = df.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, fmt='.2f', 
            square=True, linewidths=1)
plt.title('Correlation Heatmap ของ Wine Quality Dataset', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# ดู correlation กับ quality โดยเฉพาะ
quality_corr = df.corr()['quality'].sort_values(ascending=False)
print("ความสัมพันธ์ของแต่ละตัวแปรกับ Quality:")
print(quality_corr)

In [ ]:
# แสดง distribution ของ quality
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='quality', palette='viridis')
plt.title('Distribution Wine Quality', fontsize=14)
plt.xlabel('Quality Score')
plt.ylabel('Count')
plt.show()

In [ ]:
# Scatter plots ของตัวแปรที่มี correlation สูงกับ quality
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Alcohol vs Quality
sns.scatterplot(data=df, x='alcohol', y='quality', alpha=0.6, ax=axes[0,0])
axes[0,0].set_title('Alcohol vs Quality')

# Volatile Acidity vs Quality
sns.scatterplot(data=df, x='volatile acidity', y='quality', alpha=0.6, ax=axes[0,1])
axes[0,1].set_title('Volatile Acidity vs Quality')

# Sulphates vs Quality
sns.scatterplot(data=df, x='sulphates', y='quality', alpha=0.6, ax=axes[1,0])
axes[1,0].set_title('Sulphates vs Quality')

# Citric Acid vs Quality
sns.scatterplot(data=df, x='citric acid', y='quality', alpha=0.6, ax=axes[1,1])
axes[1,1].set_title('Citric Acid vs Quality')

plt.tight_layout()
plt.show()

In [ ]:
# Import libraries สำหรับ Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
# แยก Features (X) และ Target (y)
X = df.drop('quality', axis=1)
y = df['quality']

print("Shape ของ Features (X):", X.shape)
print("Shape ของ Target (y):", y.shape)
print("\nการกระจายของ Quality:")
print(y.value_counts().sort_index())

In [ ]:
# แบ่งข้อมูลเป็น Training set และ Test set (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)
print("\nการกระจายของ Quality ใน Training set:")
print(y_train.value_counts().sort_index())

In [ ]:
# Feature Scaling - สำคัญมากสำหรับ PCA
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("ข้อมูลถูก scaled แล้ว")
print("Mean ของ features ใน training set หลัง scaling:")
print(np.mean(X_train_scaled, axis=0).round(10))
print("\nStd ของ features ใน training set หลัง scaling:")
print(np.std(X_train_scaled, axis=0))

## PCA Analysis

เราจะใช้ PCA เพื่อลดจำนวนมิติของข้อมูล โดยวิเคราะห์ว่าควรใช้กี่ Principal Components

In [ ]:
# ทำ PCA เพื่อดู explained variance ของแต่ละ component
pca_full = PCA()
pca_full.fit(X_train_scaled)

# คำนวณ cumulative explained variance
explained_variance = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print("Explained Variance Ratio ของแต่ละ PC:")
for i, var in enumerate(explained_variance, 1):
    print(f"PC{i}: {var:.4f} ({cumulative_variance[i-1]:.4f} cumulative)")
    
print(f"\nจำนวน Components ที่อธิบาย 90% ของ variance: {np.argmax(cumulative_variance >= 0.90) + 1}")
print(f"จำนวน Components ที่อธิบาย 95% ของ variance: {np.argmax(cumulative_variance >= 0.95) + 1}")

In [ ]:
# Visualization: Scree Plot และ Cumulative Explained Variance
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scree Plot
axes[0].bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, color='steelblue')
axes[0].set_xlabel('Principal Component', fontsize=12)
axes[0].set_ylabel('Explained Variance Ratio', fontsize=12)
axes[0].set_title('Scree Plot', fontsize=14)
axes[0].grid(axis='y', alpha=0.3)

# Cumulative Explained Variance
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', color='green')
axes[1].axhline(y=0.90, color='r', linestyle='--', label='90% variance')
axes[1].axhline(y=0.95, color='orange', linestyle='--', label='95% variance')
axes[1].set_xlabel('Number of Components', fontsize=12)
axes[1].set_ylabel('Cumulative Explained Variance', fontsize=12)
axes[1].set_title('Cumulative Explained Variance', fontsize=14)
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ใช้ PCA เพื่อลดมิติเหลือ 2D สำหรับ visualization
pca_2d = PCA(n_components=2)
X_train_pca_2d = pca_2d.fit_transform(X_train_scaled)
X_test_pca_2d = pca_2d.transform(X_test_scaled)

# Visualization ของข้อมูลใน 2D PCA space
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_train_pca_2d[:, 0], X_train_pca_2d[:, 1], 
                     c=y_train, cmap='viridis', alpha=0.6, edgecolors='k', s=50)
plt.colorbar(scatter, label='Wine Quality')
plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
plt.title('Wine Quality Data ใน 2D PCA Space', fontsize=14)
plt.grid(alpha=0.3)
plt.show()

print(f"PC1 + PC2 อธิบาย {sum(pca_2d.explained_variance_ratio_):.2%} ของ variance")

In [ ]:
# ใช้ PCA ที่อธิบาย 95% ของ variance สำหรับ Classification
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
pca = PCA(n_components=n_components_95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

print(f"ใช้ {n_components_95} Principal Components (จาก {X_train.shape[1]} features)")
print(f"Explained Variance: {sum(pca.explained_variance_ratio_):.4f}")
print(f"\nShape หลัง PCA:")
print(f"  Training set: {X_train_pca.shape}")
print(f"  Test set: {X_test_pca.shape}")

## Classification with Random Forest

เราจะใช้ Random Forest Classifier เพื่อเปรียบเทียบผลลัพธ์ระหว่าง:
1. ข้อมูลต้นฉบับ (11 features)
2. ข้อมูลหลังผ่าน PCA

In [ ]:
# Model 1: Random Forest โดยไม่ใช้ PCA (ข้อมูลต้นฉบับ)
print("=" * 60)
print("Model 1: Random Forest without PCA")
print("=" * 60)

rf_original = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_original.fit(X_train_scaled, y_train)

# ทำนายผลลัพธ์
y_pred_original = rf_original.predict(X_test_scaled)

# คำนวณ metrics
accuracy_original = accuracy_score(y_test, y_pred_original)
precision_original = precision_score(y_test, y_pred_original, average='weighted', zero_division=0)
recall_original = recall_score(y_test, y_pred_original, average='weighted', zero_division=0)
f1_original = f1_score(y_test, y_pred_original, average='weighted', zero_division=0)

print(f"\nAccuracy: {accuracy_original:.4f}")
print(f"Precision: {precision_original:.4f}")
print(f"Recall: {recall_original:.4f}")
print(f"F1-Score: {f1_original:.4f}")

In [ ]:
# Model 2: Random Forest ด้วย PCA
print("\n" + "=" * 60)
print("Model 2: Random Forest with PCA")
print("=" * 60)

rf_pca = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_pca.fit(X_train_pca, y_train)

# ทำนายผลลัพธ์
y_pred_pca = rf_pca.predict(X_test_pca)

# คำนวณ metrics
accuracy_pca = accuracy_score(y_test, y_pred_pca)
precision_pca = precision_score(y_test, y_pred_pca, average='weighted', zero_division=0)
recall_pca = recall_score(y_test, y_pred_pca, average='weighted', zero_division=0)
f1_pca = f1_score(y_test, y_pred_pca, average='weighted', zero_division=0)

print(f"\nAccuracy: {accuracy_pca:.4f}")
print(f"Precision: {precision_pca:.4f}")
print(f"Recall: {recall_pca:.4f}")
print(f"F1-Score: {f1_pca:.4f}")

In [ ]:
# สรุปเปรียบเทียบผลลัพธ์
print("\n" + "=" * 60)
print("เปรียบเทียบผลลัพธ์")
print("=" * 60)

comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Without PCA': [accuracy_original, precision_original, recall_original, f1_original],
    'With PCA': [accuracy_pca, precision_pca, recall_pca, f1_pca],
    'Difference': [
        accuracy_pca - accuracy_original,
        precision_pca - precision_original,
        recall_pca - recall_original,
        f1_pca - f1_original
    ]
})

print(comparison_df.to_string(index=False))
print(f"\n✓ จำนวน features: {X_train.shape[1]} → {X_train_pca.shape[1]} (ลดลง {X_train.shape[1] - X_train_pca.shape[1]} features)")

## Model Evaluation

ดู Classification Report และ Confusion Matrix เพื่อวิเคราะห์ประสิทธิภาพของ Model

In [ ]:
# Classification Report สำหรับ Model without PCA
print("=" * 60)
print("Classification Report: Model WITHOUT PCA")
print("=" * 60)
print(classification_report(y_test, y_pred_original, zero_division=0))

# Classification Report สำหรับ Model with PCA
print("\n" + "=" * 60)
print("Classification Report: Model WITH PCA")
print("=" * 60)
print(classification_report(y_test, y_pred_pca, zero_division=0))

In [ ]:
# Confusion Matrix Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Confusion Matrix สำหรับ Model without PCA
cm_original = confusion_matrix(y_test, y_pred_original)
sns.heatmap(cm_original, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_title('Confusion Matrix: WITHOUT PCA', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted Quality', fontsize=12)
axes[0].set_ylabel('True Quality', fontsize=12)

# Confusion Matrix สำหรับ Model with PCA
cm_pca = confusion_matrix(y_test, y_pred_pca)
sns.heatmap(cm_pca, annot=True, fmt='d', cmap='Greens', ax=axes[1], cbar_kws={'label': 'Count'})
axes[1].set_title('Confusion Matrix: WITH PCA', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted Quality', fontsize=12)
axes[1].set_ylabel('True Quality', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance จาก Random Forest (Model without PCA)
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_original.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature Importance (Model without PCA):")
print(feature_importance)

# Visualization
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='steelblue')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance in Random Forest Model', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# PCA Components Loading - ดูว่าแต่ละ PC ประกอบด้วย features อะไรบ้าง
components_df = pd.DataFrame(
    pca.components_,
    columns=X.columns,
    index=[f'PC{i+1}' for i in range(pca.n_components_)]
)

print("PCA Components Loading (top 3 PCs):")
print(components_df.head(3).round(3))

# Heatmap ของ PCA components
plt.figure(figsize=(12, 8))
sns.heatmap(components_df, cmap='RdBu_r', center=0, annot=True, fmt='.2f', 
            cbar_kws={'label': 'Loading'}, linewidths=0.5)
plt.title('PCA Components Loading Heatmap', fontsize=14, fontweight='bold')
plt.xlabel('Original Features', fontsize=12)
plt.ylabel('Principal Components', fontsize=12)
plt.tight_layout()
plt.show()

## สรุปผลการทดลอง

### ข้อค้นพบสำคัญ:

1. **PCA Analysis**:
   - PCA ช่วยลดจำนวนมิติของข้อมูลได้อย่างมีประสิทธิภาพ
   - ใช้ Principal Components เพียงไม่กี่ตัวก็สามารถอธิบาย variance ส่วนใหญ่ได้

2. **Model Performance**:
   - Random Forest สามารถทำ classification คุณภาพไวน์ได้ดี
   - การใช้ PCA อาจช่วยลดความซับซ้อนของโมเดลโดยไม่เสีย accuracy มากนัก
   - Model มีความแม่นยำดีกับคลาสที่มีข้อมูลเยอะ (quality 5, 6) แต่อาจทำนายไม่ค่อยดีสำหรับคลาสที่มีข้อมูลน้อย

3. **Feature Importance**:
   - Features ที่สำคัญที่สุดในการทำนายคุณภาพไวน์คือ alcohol, volatile acidity, และ sulphates
   - ผลลัพธ์สอดคล้องกับ correlation analysis ที่ทำไว้ก่อนหน้านี้

4. **ข้อเสนอแนะ**:
   - ควรลองใช้เทคนิคจัดการข้อมูล imbalanced เช่น SMOTE หรือ class_weight
   - อาจลองใช้ algorithms อื่นๆ เช่น SVM, XGBoost, Neural Networks
   - ทำ Hyperparameter Tuning เพื่อเพิ่มประสิทธิภาพของโมเดล